# Phased Array Systems Tutorial

**A comprehensive guide to designing and optimizing phased array antenna systems**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/phased-array-systems/blob/main/notebooks/tutorial_phased_array_trade_study.ipynb)

This tutorial demonstrates how to:
1. Define phased array architectures
2. Set up communications link scenarios
3. Specify system requirements
4. Evaluate single design points
5. Run Design of Experiments (DOE) trade studies
6. Extract Pareto-optimal designs
7. Visualize trade spaces

## Installation

First, install the `phased-array-systems` package:

In [ ]:
# Install the package (uncomment for Colab)
# !pip install phased-array-systems matplotlib

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt

from phased_array_systems.architecture import (
    Architecture,
    ArrayConfig,
    RFChainConfig,
    CostConfig,
)
from phased_array_systems.scenarios import CommsLinkScenario
from phased_array_systems.requirements import Requirement, RequirementSet
from phased_array_systems.evaluate import evaluate_case, evaluate_case_with_report

print("Imports successful!")

## Part 1: Defining a Phased Array Architecture

A phased array system is characterized by:
- **Array geometry**: Number of elements, spacing, geometry type
- **RF chain**: Transmit power, efficiency, noise figure
- **Cost**: Per-element cost, NRE, integration costs

In [ ]:
# Define a basic 8x8 rectangular array
array_config = ArrayConfig(
    geometry="rectangular",
    nx=8,                    # 8 elements in x-direction
    ny=8,                    # 8 elements in y-direction
    dx_lambda=0.5,           # Half-wavelength spacing in x
    dy_lambda=0.5,           # Half-wavelength spacing in y
    scan_limit_deg=60.0,     # Maximum scan angle
)

print(f"Array: {array_config.nx} x {array_config.ny} = {array_config.n_elements} elements")
print(f"Geometry: {array_config.geometry}")
print(f"Element spacing: {array_config.dx_lambda}λ x {array_config.dy_lambda}λ")

In [ ]:
# Define the RF chain
rf_config = RFChainConfig(
    tx_power_w_per_elem=1.0,   # 1 Watt per element
    pa_efficiency=0.3,          # 30% PA efficiency
    noise_figure_db=3.0,        # 3 dB noise figure
    feed_loss_db=1.0,           # 1 dB feed network loss
)

print(f"TX power per element: {rf_config.tx_power_w_per_elem} W")
print(f"Total TX power: {rf_config.tx_power_w_per_elem * array_config.n_elements} W")
print(f"PA efficiency: {rf_config.pa_efficiency * 100}%")

In [ ]:
# Define cost parameters
cost_config = CostConfig(
    cost_per_elem_usd=100.0,    # $100 per element
    nre_usd=10000.0,            # $10k non-recurring engineering
    integration_cost_usd=5000.0 # $5k integration
)

total_cost = (
    cost_config.cost_per_elem_usd * array_config.n_elements +
    cost_config.nre_usd +
    cost_config.integration_cost_usd
)
print(f"Estimated total cost: ${total_cost:,.0f}")

In [ ]:
# Combine into a complete architecture
architecture = Architecture(
    array=array_config,
    rf=rf_config,
    cost=cost_config,
    name="8x8 X-band Array"
)

print(f"Architecture: {architecture.name}")
print(f"Total elements: {architecture.n_elements}")

## Part 2: Defining a Communications Scenario

The scenario defines the operating conditions:
- Frequency and bandwidth
- Link range
- Required SNR for demodulation
- Environmental factors

In [ ]:
# Define a communications link scenario
scenario = CommsLinkScenario(
    freq_hz=10e9,           # 10 GHz (X-band)
    bandwidth_hz=10e6,      # 10 MHz bandwidth
    range_m=100e3,          # 100 km range
    required_snr_db=10.0,   # 10 dB required SNR
    scan_angle_deg=0.0,     # Boresight pointing
    rx_antenna_gain_db=0.0, # Isotropic receive antenna
    rx_noise_temp_k=290.0,  # Room temperature
)

print(f"Frequency: {scenario.freq_hz / 1e9:.1f} GHz")
print(f"Wavelength: {scenario.wavelength_m * 1000:.1f} mm")
print(f"Range: {scenario.range_m / 1e3:.0f} km")
print(f"Required SNR: {scenario.required_snr_db} dB")

## Part 3: Defining Requirements

Requirements define the pass/fail criteria for a design:
- **Must** requirements: Design fails if not met
- **Should** requirements: Desired but not critical

In [ ]:
# Define system requirements
requirements = RequirementSet(
    requirements=[
        Requirement(
            id="REQ-001",
            name="Minimum EIRP",
            metric_key="eirp_dbw",
            op=">=",
            value=40.0,
            units="dBW",
            severity="must",
        ),
        Requirement(
            id="REQ-002",
            name="Positive Link Margin",
            metric_key="link_margin_db",
            op=">=",
            value=0.0,
            units="dB",
            severity="must",
        ),
        Requirement(
            id="REQ-003",
            name="Maximum Cost",
            metric_key="cost_usd",
            op="<=",
            value=50000.0,
            units="USD",
            severity="must",
        ),
        Requirement(
            id="REQ-004",
            name="Preferred Link Margin",
            metric_key="link_margin_db",
            op=">=",
            value=6.0,
            units="dB",
            severity="should",
        ),
    ],
    name="Comms Array Requirements"
)

print(f"Defined {len(requirements)} requirements:")
for req in requirements:
    print(f"  [{req.severity.upper()}] {req.id}: {req.name} ({req.metric_key} {req.op} {req.value})")

## Part 4: Single Case Evaluation

Evaluate the architecture against the scenario and verify requirements.

In [ ]:
# Evaluate the design
metrics, report = evaluate_case_with_report(
    architecture, 
    scenario, 
    requirements
)

print("=" * 60)
print("EVALUATION RESULTS")
print("=" * 60)

print("\nAntenna Metrics:")
print(f"  Peak Gain: {metrics['g_peak_db']:.2f} dB")
print(f"  Directivity: {metrics['directivity_db']:.2f} dB")
print(f"  Beamwidth (Az/El): {metrics['beamwidth_az_deg']:.2f}° / {metrics['beamwidth_el_deg']:.2f}°")
print(f"  Sidelobe Level: {metrics['sll_db']:.2f} dB")

print("\nLink Budget:")
print(f"  EIRP: {metrics['eirp_dbw']:.2f} dBW")
print(f"  Path Loss: {metrics['path_loss_db']:.2f} dB")
print(f"  RX Power: {metrics['rx_power_dbw']:.2f} dBW")
print(f"  SNR: {metrics['snr_rx_db']:.2f} dB")
print(f"  Link Margin: {metrics['link_margin_db']:.2f} dB")

print("\nSWaP-C:")
print(f"  RF Power: {metrics['rf_power_w']:.1f} W")
print(f"  Prime Power: {metrics['prime_power_w']:.1f} W")
print(f"  Total Cost: ${metrics['cost_usd']:,.0f}")

In [ ]:
# Check requirements verification
print("\n" + "=" * 60)
print("REQUIREMENTS VERIFICATION")
print("=" * 60)

status = "✓ PASS" if report.passes else "✗ FAIL"
print(f"\nOverall Status: {status}")
print(f"Must Requirements: {report.must_pass_count}/{report.must_total_count} passed")
print(f"Should Requirements: {report.should_pass_count}/{report.should_total_count} passed")

print("\nDetailed Results:")
for result in report.results:
    req = result.requirement
    status_str = "✓" if result.passes else "✗"
    if result.actual_value is not None:
        print(f"  {status_str} {req.id}: {result.actual_value:.2f} {req.op} {req.value} (margin: {result.margin:+.2f})")

## Part 5: Design of Experiments (DOE) Trade Study

Now let's explore the design space by varying key parameters and finding optimal trade-offs.

In [ ]:
# Import trade study tools
from phased_array_systems.trades import (
    DesignSpace,
    generate_doe,
    BatchRunner,
    filter_feasible,
    extract_pareto,
    rank_pareto,
)

print("Trade study tools imported!")

In [ ]:
# Define the design space to explore
design_space = (
    DesignSpace(name="Comms Array Trade Space")
    # Variable parameters (array dimensions must be powers of 2 for RF constraints)
    .add_variable("array.nx", type="categorical", values=[4, 8, 16])
    .add_variable("array.ny", type="categorical", values=[4, 8, 16])
    .add_variable("rf.tx_power_w_per_elem", type="float", low=0.5, high=3.0)
    .add_variable("cost.cost_per_elem_usd", type="float", low=75.0, high=150.0)
    # Fixed parameters
    .add_variable("array.geometry", type="categorical", values=["rectangular"])
    .add_variable("array.dx_lambda", type="float", low=0.5, high=0.5)
    .add_variable("array.dy_lambda", type="float", low=0.5, high=0.5)
    .add_variable("array.scan_limit_deg", type="float", low=60.0, high=60.0)
    .add_variable("array.enforce_subarray_constraint", type="categorical", values=[True])
    .add_variable("rf.pa_efficiency", type="float", low=0.3, high=0.3)
    .add_variable("rf.noise_figure_db", type="float", low=3.0, high=3.0)
    .add_variable("rf.n_tx_beams", type="int", low=1, high=1)
    .add_variable("rf.feed_loss_db", type="float", low=1.0, high=1.0)
    .add_variable("rf.system_loss_db", type="float", low=0.0, high=0.0)
)

print(f"Design space: {design_space.n_dims} variables")

In [ ]:
# Generate DOE using Latin Hypercube Sampling
n_samples = 100
doe = generate_doe(design_space, method="lhs", n_samples=n_samples, seed=42)

print(f"Generated {len(doe)} design cases")
print("\nSample of design variables:")
doe[["array.nx", "array.ny", "rf.tx_power_w_per_elem"]].head()

In [ ]:
# Run batch evaluation
print("Running batch evaluation...")

runner = BatchRunner(scenario, requirements)

def progress_callback(completed, total):
    if completed % 25 == 0 or completed == total:
        print(f"  Progress: {completed}/{total} ({completed/total*100:.0f}%)")

results = runner.run(doe, n_workers=1, progress_callback=progress_callback)

print(f"\nCompleted {len(results)} evaluations")

In [ ]:
# Filter to feasible designs
feasible = filter_feasible(results, requirements)
print(f"Feasible designs: {len(feasible)} / {len(results)} ({len(feasible)/len(results)*100:.1f}%)")

## Part 6: Pareto Analysis

Find the Pareto-optimal trade-off between cost and performance.

In [ ]:
# Extract Pareto frontier: minimize cost, maximize EIRP
pareto = extract_pareto(
    feasible,
    objectives=[
        ("cost_usd", "minimize"),
        ("eirp_dbw", "maximize"),
    ]
)

print(f"Pareto-optimal designs: {len(pareto)}")

In [ ]:
# Rank Pareto designs with equal weights
ranked = rank_pareto(
    pareto,
    objectives=[
        ("cost_usd", "minimize"),
        ("eirp_dbw", "maximize"),
    ],
    weights=[0.5, 0.5],
)

print("Top 5 Pareto-optimal designs:")
print("-" * 80)
for i, (_, row) in enumerate(ranked.head(5).iterrows()):
    print(f"{i+1}. Array: {int(row['array.nx'])}x{int(row['array.ny'])} ({int(row['n_elements'])} elem) | "
          f"Cost: ${row['cost_usd']:,.0f} | EIRP: {row['eirp_dbw']:.1f} dBW | "
          f"Margin: {row['link_margin_db']:.1f} dB")

## Part 7: Visualization

Visualize the trade space to understand design trade-offs.

In [ ]:
# Create Pareto plot: Cost vs EIRP
fig, ax = plt.subplots(figsize=(10, 7))

# Plot infeasible designs
infeasible = results[results["verification.passes"] != 1.0]
ax.scatter(
    infeasible["cost_usd"], 
    infeasible["eirp_dbw"],
    c="lightgray", s=30, alpha=0.5, marker="x", label="Infeasible"
)

# Plot feasible designs
scatter = ax.scatter(
    feasible["cost_usd"],
    feasible["eirp_dbw"],
    c=feasible["link_margin_db"],
    cmap="viridis",
    s=60,
    alpha=0.7,
    label="Feasible"
)

# Highlight Pareto front
ax.scatter(
    pareto["cost_usd"],
    pareto["eirp_dbw"],
    facecolors="none",
    edgecolors="red",
    s=150,
    linewidths=2,
    label="Pareto Optimal"
)

# Draw Pareto line
sorted_pareto = pareto.sort_values("cost_usd")
ax.plot(sorted_pareto["cost_usd"], sorted_pareto["eirp_dbw"], "r--", alpha=0.5, linewidth=1.5)

# Formatting
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Link Margin (dB)")

ax.set_xlabel("Total Cost (USD)", fontsize=12)
ax.set_ylabel("EIRP (dBW)", fontsize=12)
ax.set_title("Phased Array Trade Space: Cost vs EIRP", fontsize=14)
ax.legend(loc="lower right")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot array size distribution on Pareto front
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cost vs Number of Elements
ax1 = axes[0]
scatter1 = ax1.scatter(
    feasible["n_elements"],
    feasible["cost_usd"],
    c=feasible["eirp_dbw"],
    cmap="plasma",
    s=50,
    alpha=0.7
)
ax1.scatter(
    pareto["n_elements"],
    pareto["cost_usd"],
    facecolors="none",
    edgecolors="green",
    s=120,
    linewidths=2
)
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label("EIRP (dBW)")
ax1.set_xlabel("Number of Elements")
ax1.set_ylabel("Total Cost (USD)")
ax1.set_title("Cost vs Array Size")
ax1.grid(True, alpha=0.3)

# EIRP vs Number of Elements
ax2 = axes[1]
scatter2 = ax2.scatter(
    feasible["n_elements"],
    feasible["eirp_dbw"],
    c=feasible["prime_power_w"],
    cmap="coolwarm",
    s=50,
    alpha=0.7
)
ax2.scatter(
    pareto["n_elements"],
    pareto["eirp_dbw"],
    facecolors="none",
    edgecolors="green",
    s=120,
    linewidths=2
)
cbar2 = plt.colorbar(scatter2, ax=ax2)
cbar2.set_label("Prime Power (W)")
ax2.set_xlabel("Number of Elements")
ax2.set_ylabel("EIRP (dBW)")
ax2.set_title("EIRP vs Array Size")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Show the best compromise design
best = ranked.iloc[0]

print("=" * 60)
print("RECOMMENDED DESIGN (Best Compromise)")
print("=" * 60)
print(f"\nArray Configuration:")
print(f"  Geometry: {best['array.geometry']}")
print(f"  Size: {int(best['array.nx'])} x {int(best['array.ny'])} = {int(best['n_elements'])} elements")
print(f"  Element spacing: {best['array.dx_lambda']}λ")

print(f"\nRF Chain:")
print(f"  TX power/element: {best['rf.tx_power_w_per_elem']:.2f} W")
print(f"  Total RF power: {best['rf_power_w']:.1f} W")

print(f"\nPerformance:")
print(f"  EIRP: {best['eirp_dbw']:.1f} dBW")
print(f"  Link Margin: {best['link_margin_db']:.1f} dB")
print(f"  Peak Gain: {best['g_peak_db']:.1f} dB")

print(f"\nSWaP-C:")
print(f"  Prime Power: {best['prime_power_w']:.0f} W")
print(f"  Total Cost: ${best['cost_usd']:,.0f}")

## Summary

This tutorial demonstrated how to:

1. **Define architectures** using Pydantic models for arrays, RF chains, and cost
2. **Set up scenarios** with frequency, range, and link requirements
3. **Specify requirements** with must/should severity levels
4. **Evaluate single designs** and verify requirement compliance
5. **Run DOE trade studies** using Latin Hypercube Sampling
6. **Extract Pareto frontiers** to find optimal trade-offs
7. **Visualize results** to understand the design space

### Next Steps

- Try different DOE methods (`grid`, `random`, `lhs`)
- Explore radar scenarios with `RadarDetectionScenario`
- Add more design variables to the trade study
- Export results with `export_results()` for further analysis

For more information, see the [documentation](https://jman4162.github.io/phased-array-systems).